In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read Echonest Taste profile dataset
#taste = pd.read_csv('./data/train_triplets.txt', sep='\t', header=None, names=['user_id','song_id','play_count'])

In [3]:
#taste.head()

In [4]:
#taste.shape

In [5]:
import os
import pyspark

In [6]:
from pyspark import SparkContext
sc = SparkContext()

In [7]:
taste_file = os.path.join('.','data','subset_taste_profile.csv')
taste_raw_data = sc.textFile(taste_file)

In [8]:
taste_raw_data

./data/subset_taste_profile.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
taste_data = taste_raw_data.map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [10]:
taste_data.take(5)

[(u'user_index', u'song_index', u'play_count'),
 (u'0', u'0', u'1'),
 (u'0', u'1', u'1'),
 (u'0', u'2', u'2'),
 (u'0', u'3', u'1')]

In [11]:
#from pyspark.sql import SQLContext, Row
#sqlContext = SQLContext(sc)
#taste_rows = taste_data.map(lambda x: Row(user_id=x[0], song_id=x[1], play_count=int(x[2])))
#taste_dataframe = sqlContext.createDataFrame(taste_data, ["user_id", "song_id", "play_count"])

In [12]:
#taste_dataframe.take(5)

In [13]:
# StringIndexer 
#from pyspark.ml.feature import StringIndexer

#userIndex = StringIndexer(inputCol="user_id", outputCol="user_index")
#user_ID = userIndex.fit(taste_dataframe).transform(taste_dataframe)

In [14]:
#user_ID.show(10)

In [15]:
training_RDD, validation_RDD, test_RDD = taste_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [17]:
# Model selection (ALS parameter tuning)
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 7.22033451369
For rank 8 the RMSE is 7.21919181814
For rank 12 the RMSE is 7.21842479653
The best model was trained with rank 12


In [19]:
predictions.take(3)

[((1356, 23776), 0.0011338368332428949),
 ((893, 23776), 0.012175205875991333),
 ((6598, 23776), 0.0047738702913099436)]